In [4]:
import pandas as pd
import numpy as np
from scipy import sparse
import optuna
from matplotlib import pyplot as plt
import time

import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

In [12]:
class StackOverflowDataset(Dataset):
    def __init__(self, filename) -> None:
        super().__init__()
        self.data = pd.read_parquet(filename)

        self.title = self.data['Title']
        self.body = self.data['Body']
        self.tags = self.data['Tags']
        self.target = self.data['target'] if 'target' in self.data.columns else None

        def split_tags(text):
            return ' '.join(text[1: -1].split('><'))
        self.tags = self.tags.apply(lambda x: split_tags(x))

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        data = self.title.iloc[idx] + self.body.iloc[idx] + self.tags.iloc[idx]
        if self.target is not None:
            return data, self.target.iloc[idx]
        else:
            return data

In [13]:
train_iter = StackOverflowDataset('./data/train.parquet')

In [15]:
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [16]:
text_pipeline = lambda x: vocab(tokenizer(x))

In [84]:
device = torch.device("cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
         label_list.append(_label)
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = StackOverflowDataset('./data/train.parquet')
dataloader = DataLoader(train_iter, batch_size=32, collate_fn=collate_batch)

In [101]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        linear_output = self.fc(embedded)
        return self.fc2(linear_output)

In [102]:
train_iter = StackOverflowDataset('./data/train.parquet')
num_class = len(set([label for (_, label) in train_iter]))
vocab_size = len(vocab)
emsize = 64
hidden_dim = 32
model = TextClassificationModel(vocab_size, emsize, hidden_dim, num_class).to(device)

In [103]:
def train(dataloader, optimizer, epoch, criterion):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader, criterion):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [105]:
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter = StackOverflowDataset('./data/train.parquet')
test_iter = StackOverflowDataset('./data/test.parquet')
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, optimizer, epoch, criterion)
    accu_val = evaluate(valid_dataloader, criterion)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/  713 batches | accuracy    0.799
-----------------------------------------------------------
| end of epoch   1 | time: 17.86s | valid accuracy    0.853 
-----------------------------------------------------------
| epoch   2 |   500/  713 batches | accuracy    0.839
-----------------------------------------------------------
| end of epoch   2 | time: 19.53s | valid accuracy    0.860 
-----------------------------------------------------------
| epoch   3 |   500/  713 batches | accuracy    0.862
-----------------------------------------------------------
| end of epoch   3 | time: 19.39s | valid accuracy    0.852 
-----------------------------------------------------------


KeyboardInterrupt: 

In [98]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

prediction = [predict(text, text_pipeline) for text in test_dataset]

In [100]:
prediction

[3,
 1,
 1,
 1,
 3,
 1,
 2,
 3,
 1,
 2,
 1,
 3,
 3,
 3,
 1,
 3,
 3,
 2,
 3,
 1,
 1,
 3,
 3,
 1,
 2,
 2,
 2,
 3,
 2,
 3,
 1,
 3,
 1,
 3,
 2,
 3,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 3,
 2,
 3,
 1,
 1,
 3,
 3,
 3,
 2,
 1,
 1,
 3,
 3,
 2,
 3,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 3,
 1,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 1,
 2,
 1,
 2,
 3,
 1,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 3,
 2,
 1,
 3,
 3,
 1,
 3,
 2,
 3,
 1,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 2,
 1,
 2,
 3,
 2,
 2,
 3,
 2,
 3,
 3,
 2,
 1,
 2,
 3,
 3,
 1,
 2,
 3,
 3,
 3,
 3,
 1,
 2,
 2,
 2,
 2,
 1,
 3,
 3,
 2,
 1,
 2,
 3,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 1,
 2,
 3,
 3,
 1,
 3,
 1,
 3,
 3,
 1,
 2,
 1,
 1,
 1,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 2,
 1,
 1,
 2,
 3,
 3,
 3,
 2,
 3,
 2,
 1,
 3,
 1,
 3,
 2,
 3,
 2,
 1,
 3,
 1,
 1,
 1,
 3,
 1,
 2,
 3,
 2,
 2,
 1,
 2,
 2,
 3,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 3,
 2,
 1,
 1,
 1,
 3,
